In [40]:
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV

import numpy as np
import time
import pandas as pd
import os

In [41]:
is_vae = False

In [42]:
# Just choose the name of the dataset directory
DATA_DIR = '/Users/tomas/Documents/FEUP/Tese/data/ml-20m/processed_70_10_20'
if is_vae:
    PARSE_DATA_DIR = os.path.join(DATA_DIR, 'embeddings/vae')
else:
    PARSE_DATA_DIR = os.path.join(DATA_DIR, 'embeddings/cdae')

In [43]:
file = 'metadataset_k_20.csv'

In [44]:
#read in the data using pandas
metadataset = pd.read_csv(os.path.join(PARSE_DATA_DIR, file ))
metadataset.head()

,original_id,f0,f1,f2,f3,f4,f5,f6,f7,f8,...,f41,f42,f43,f44,f45,f46,f47,f48,f49,first_place
0,1,2.092112e-32,6.249415e-32,-6.000611e-33,5.893090e-32,4.020546e-32,2.152657e-32,-5.836300e-32,2.339763e-32,4.832709e-32,...,1.450966e-33,-2.433296e-32,-1.128275e-33,-1.185880e-33,1.241357e-32,-6.068290e-33,3.252277e-32,2.221478e-32,9.671203e-34,zeroes
1,2,-1.855838e-32,-2.090192e-32,-1.898156e-32,2.504322e-32,-2.451386e-32,-6.685095e-34,-3.221332e-32,-4.198181e-32,-6.362588e-32,...,1.600629e-32,3.991929e-32,-1.531876e-32,-4.649628e-32,-4.278011e-32,2.160474e-32,-3.867348e-32,2.380331e-32,-2.335757e-32,als_ndcg
2,3,-1.169303e-32,-6.049711e-32,4.136449e-32,-9.663286e-33,-1.924273e-32,3.455974e-32,1.667681e-32,5.966751e-34,-2.579415e-32,...,5.994075e-33,-1.462464e-32,-1.664031e-32,2.628347e-33,-1.029011e-32,1.240914e-32,-1.170141e-32,-2.649629e-32,2.516531e-32,most_popular_ndcg
3,4,-2.586232e-32,-3.247477e-32,6.048788e-33,1.627761e-32,5.529978e-32,-1.838027e-33,1.555958e-32,5.057206e-32,-2.870304e-33,...,1.395353e-32,-2.553700e-32,1.501834e-32,4.594490e-32,1.732268e-32,1.482250e-32,-7.583220e-33,4.964787e-32,1.042384e-32,zeroes
4,5,-1.429949e-32,-1.369787e-32,-2.009418e-32,4.561260e-32,1.526659e-32,-2.487647e-32,5.173627e-33,-3.500297e-32,4.993383e-32,...,6.116732e-33,-1.546610e-32,-1.768405e-32,1.836809e-32,2.371812e-32,-1.883063e-32,5.819227e-32,4.135692e-32,-4.840399e-32,most_popular_ndcg


In [46]:
#---- SET INPUTS -----
#remove the original_id and the target (first_place)
scaler = StandardScaler()
#Compute the mean and std to be used for later scaling.
scaler.fit(metadataset.drop(columns=['first_place','original_id']))
# Perform standardization by centering and scaling
inputs_transform = scaler.transform(metadataset.drop(columns=['first_place','original_id']))
inputs = pd.DataFrame(inputs_transform)
inputs.head()

,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
0,-0.024315,-0.02473,-0.024062,-0.024096,-0.024189,-0.024405,-0.024405,-0.024232,-0.024043,-0.024621,...,-0.024586,-0.024683,-0.02487,-0.024779,-0.024216,-0.024449,-0.024781,-0.024835,-0.024593,-0.024418
1,-0.024315,-0.02473,-0.024062,-0.024096,-0.024189,-0.024405,-0.024405,-0.024232,-0.024043,-0.024621,...,-0.024586,-0.024683,-0.02487,-0.024779,-0.024216,-0.024449,-0.024781,-0.024835,-0.024593,-0.024418
2,-0.024315,-0.02473,-0.024062,-0.024096,-0.024189,-0.024405,-0.024405,-0.024232,-0.024043,-0.024621,...,-0.024586,-0.024683,-0.02487,-0.024779,-0.024216,-0.024449,-0.024781,-0.024835,-0.024593,-0.024418
3,-0.024315,-0.02473,-0.024062,-0.024096,-0.024189,-0.024405,-0.024405,-0.024232,-0.024043,-0.024621,...,-0.024586,-0.024683,-0.02487,-0.024779,-0.024216,-0.024449,-0.024781,-0.024835,-0.024593,-0.024418
4,-0.024315,-0.02473,-0.024062,-0.024096,-0.024189,-0.024405,-0.024405,-0.024232,-0.024043,-0.024621,...,-0.024586,-0.024683,-0.02487,-0.024779,-0.024216,-0.024449,-0.024781,-0.024835,-0.024593,-0.024418


1256

In [60]:
#---- SET TARGET -----
target = metadataset['first_place'].values
target

array(['zeroes', 'als_ndcg', 'most_popular_ndcg', ..., 'bpr_ndcg',
       'bpr_ndcg', 'most_popular_ndcg'], dtype=object)

In [61]:
#create a new KNN model
knn_cv = KNeighborsClassifier(n_neighbors=48)

In [62]:
#train model with cv of 5 
cv_scores = cross_val_score(knn_cv, inputs, target, cv=5)

In [63]:
#print each cv score (accuracy) and average them
#print(cv_scores)
print('cv_scores mean:{}'.format(np.mean(cv_scores)))

cv_scores mean:0.2221769949111954


In [64]:
#create new a knn model
knn2 = KNeighborsClassifier()

In [65]:
param_grid = {'n_neighbors': np.arange(1, 50)}

In [66]:
#use gridsearch to test all values for n_neighbors
knn_gscv = GridSearchCV(knn2, param_grid, cv=5)

In [ ]:
start_train = time.time()
knn_gscv.fit(inputs, target)
print("Training took", time.time() - start_train)

In [ ]:
print('o---------la')

In [ ]:
knn_gscv.best_params_